# Lab | Customer Analysis Final Round


For this lab, we still keep using the marketing_customer_analysis.csv file that you can find in the files_for_lab folder.

It's time to put it all together. Remember the previous rounds and follow the steps as shown in previous lectures.

## 01 - Problem (case study)

- Data Description.

- Goal.

In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

## 02 - Getting Data

Read the .csv file.

## 03 - Cleaning/Wrangling/EDA

- Change headers names.
- Deal with NaN values.
- Categorical Features.
- Numerical Features.
- Exploration.

## 04 - Processing Data

- Dealing with outliers.
- Normalization.
- Encoding Categorical Data.
- Splitting into train set and test set.

## 05 - Modeling

Apply model.

## 06 - Model Validation

- R2.
- MSE.
- RMSE.
- MAE.

## 07 - Reporting

Present results.